In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dir = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumor Training'
test_dir = '/Users/yash/Downloads/BrainTumourdetection/brain tumour dataset/Brain Tumour Testing'

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only rescaling for the testing set
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)


Found 4600 images belonging to 2 classes.
Found 1872 images belonging to 2 classes.


In [4]:
# Load pre-trained VGG-16 model without the top (fully connected) layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False


In [5]:
model = Sequential()

# Add VGG-16 base model
model.add(base_model)

# Flatten the output and add a dense layer
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Epoch 1/10
143/143 [==============================] - 818s 6s/step - loss: 0.5020 - accuracy: 0.8065 - val_loss: 0.1767 - val_accuracy: 0.9251
Epoch 2/10
143/143 [==============================] - 878s 6s/step - loss: 0.1746 - accuracy: 0.9332 - val_loss: 0.0938 - val_accuracy: 0.9666
Epoch 3/10
143/143 [==============================] - 943s 7s/step - loss: 0.1272 - accuracy: 0.9518 - val_loss: 0.1373 - val_accuracy: 0.9418
Epoch 4/10
143/143 [==============================] - 1037s 7s/step - loss: 0.1151 - accuracy: 0.9523 - val_loss: 0.0580 - val_accuracy: 0.9833
Epoch 5/10
143/143 [==============================] - 975s 7s/step - loss: 0.0887 - accuracy: 0.9665 - val_loss: 0.0515 - val_accuracy: 0.9817
Epoch 6/10
143/143 [==============================] - 966s 7s/step - loss: 0.0885 - accuracy: 0.9661 - val_loss: 0.0398 - val_accuracy: 0.9871
Epoch 7/10
143/143 [==============================] - 977s 7s/step - loss: 0.0718 - accuracy: 0.9737 - val_loss: 0.0568 - val_accuracy: 0.977

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
y_true = test_generator.classes
y_pred_raw = model.predict(test_generator)
y_pred = (y_pred_raw > 0.5).astype(int)  # Convert probabilities to binary predictions

# Calculate accuracy, precision, recall, and confusion matrix
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall (Sensitivity): {recall:.4f}')
print('Confusion Matrix:')
print(conf_matrix)

59/59 [==============================] - 315s 5s/step
Accuracy: 0.5118
Precision: 0.5520
Recall (Sensitivity): 0.5520
Confusion Matrix:
[[395 457]
 [457 563]]


In [8]:
import tensorflow as tf
from keras.models import load_model

model.save('Brain Tumour.h5')

In [9]:
from keras.models import load_model

In [10]:
from keras import utils

In [11]:
from keras.applications.vgg16 import preprocess_input


In [12]:
import numpy as np

In [13]:
model=load_model('Brain Tumour.h5')

In [14]:
img_path = '/Users/yash/Downloads/project /Dataset/Testing/no_tumor/image(104).jpg'  # Change this to the path of your brain image
img = utils.load_img(img_path, target_size=(224, 224))
img_array = utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)


In [15]:
prediction = model.predict(img_array)
prediction

1/1 [==============================] - 0s 179ms/step


array([[0.]], dtype=float32)

In [16]:
if prediction[0][0] >= 0.5:
         print("Tumour detected")
else:
        print("Result is Normal")

Result is Normal
